In [1]:
!pip install -qU transformers datasets accelerate tqdm wandb git+https://github.com/huggingface/diffusers

['wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc']

In [2]:
!rm -rf lora_out

!python t2i_lora.py \
    --pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5 \
    --dataset_name=naxalpha/stable-icons-128 \
    --resolution=128 \
    --report_to=wandb \
    --caption_column="tags" \
    --image_column="image" \
    --validation_prompt="bw-icon" \
    --output_dir="lora_out" \
    --seed=1000 \
    --train_batch_size=32

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:243: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
03/19/2023 09:45:35 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

{'clip_sample_range', 'prediction_type', 'variance_type', 'sample_max_value', 'thresholding', 'dynamic_thresholding_ratio'} was not found in config. Values will be initialized to default values.
{'scaling_factor'} was not found in config. Values will be initialized to default values.
{'dual_cross_attention', 'resnet_time_scale_shift', 'projection_class_embeddings_input_dim', 'use_linear_projection', 'time_embedding_type', 'time_cond_proj_dim', 'timestep_post_act', 'class_embed_type', 'num_class_embeds', 'conv_out_kernel', 'conv_in_kernel', 'upcast_attention', 'only_cross_attention', 'mid_block_type'} was not found 

In [3]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy

/opt/conda/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [4]:
prompts = [
    "bw-icon, download, outline, square, arrow down, file",
    "bw-icon, home, thin",
    "a black and white icon of home",
    "bw-icon, hand, drop, water",
    "bw-icon, perfect square",
    "bw-icon, google, logo",
    "brand, facebook, slim",
    "bw-icon, fast",
    "bw-icon, glasses",
    "bw-icon, water droplet",
    "bw-icon, mountain",
    "bw-icon, hand holding a drop of water",
    "bw-icon, 3d",
    "bw-icon, flat earth",
    "bw-icon, super fun",
    "bw-icon",
]
spp = 16

In [5]:
import wandb

wandb.init(project='lora-progress')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: naxalpha. Use `wandb login --relogin` to force relogin


In [7]:
from torchvision.utils import make_grid
from torchvision.transforms.functional import to_tensor, to_pil_image

for i in range(1, 34):
    j = i * 500
    k = f'lora_out/checkpoint-{j}'
    pipe.unet.load_attn_procs(k, weight_name="pytorch_model.bin")
    
    imgs = pipe(prompts, width=128, height=128, num_images_per_prompt=spp).images
    imgs = to_pil_image(make_grid(list(map(to_tensor, imgs)), nrow=spp))
    
    wandb.log({'img': wandb.Image(imgs)}, step=i)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

OSError: lora_out/checkpoint-13000 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login`.

In [ ]:
wandb.finish()